In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from getchinamap.getchinamap import DownloadChmap
import rasterio.mask
import cartopy.crs as ccrs
import matplotlib as mpl
import datetime
dl_engine = DownloadChmap()


In [2]:

dem_raw_data = rasterio.open("数据集/ChinaDEM1km_WGS84.tif")
# dem_raw_data = rasterio.open("数据集/Global_DEM/World_e-Atlas-UCSD_SRTM30-plus_v8.tif")


In [3]:
dem_raw_data.crs

ERROR 1: PROJ: proj_identify: /opt/anaconda3/envs/devc/share/proj/proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.


CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]')

In [4]:
dem_raw_data.bounds

BoundingBox(left=53.46698648391842, bottom=10.408439857763582, right=152.87245373475398, top=59.554459059540065)

In [5]:
dem_raw_data.transform

Affine(0.00912646596133268, 0.0, 53.46698648391842,
       0.0, -0.00912646596133268, 59.554459059540065)

In [6]:
%matplotlib

Using matplotlib backend: MacOSX


In [7]:
# china_boundary_gdf = dl_engine.download_country(target='边界')
province_name = '西藏自治区'
china_boundary_gdf = dl_engine.download_province(province_name=province_name,target='边界')
china_boundary_gdf_sub = dl_engine.download_province(province_name=province_name,target='市')
# china_boundary_gdf = dl_engine.download_country(target='边界')
# china_boundary_gdf_sub = dl_engine.download_country(target='省')
china_boundary_gdf_sub

,adcode,name,childrenNum,level,parent,subFeatureIndex,geometry
0,540100,拉萨市,8,city,{'adcode': 540000},0,"MULTIPOLYGON (((90.34923 29.30736, 90.37761 29..."
1,540200,日喀则市,18,city,{'adcode': 540000},1,"MULTIPOLYGON (((89.89917 29.97530, 89.88384 29..."
2,540300,昌都市,11,city,{'adcode': 540000},2,"MULTIPOLYGON (((98.68483 28.43523, 98.68939 28..."
3,540400,林芝市,7,city,{'adcode': 540000},3,"MULTIPOLYGON (((94.15421 30.48689, 94.15130 30..."
4,540500,山南市,12,city,{'adcode': 540000},4,"MULTIPOLYGON (((90.34923 29.30736, 90.33824 29..."
5,540600,那曲市,11,city,{'adcode': 540000},5,"MULTIPOLYGON (((94.75946 32.52937, 94.75781 32..."
6,542500,阿里地区,7,city,{'adcode': 540000},6,"MULTIPOLYGON (((86.07057 35.86067, 86.05979 35..."


In [8]:
out_image, out_transform = rasterio.mask.mask(dem_raw_data, china_boundary_gdf['geometry'], crop=True)
out_meta = dem_raw_data.meta
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

In [9]:
clean_data = out_image.copy().astype('float')
clean_data = clean_data[0, :, :]
clean_data[clean_data == out_meta['nodata']] = np.nan
# clean_data[clean_data <= -200] = np.nan

# clean_data

## 全部都是ccrs.PlateCarree()

In [10]:
mat_extent = [getattr(out_transform, 'c'),
              getattr(out_transform, 'c') + clean_data.shape[1] * getattr(out_transform, 'a'),
              getattr(out_transform, 'f') + clean_data.shape[0] * getattr(out_transform, 'e'),
              getattr(out_transform, 'f')]

colors = ["#33A02C", "#B2DF8A", "#FDBF6F", "#1F78B4", "#999999", "#E31A1C", "#E6E6E6", "#A6CEE3"]

myccrs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(10, 5), dpi=150, subplot_kw={'projection': myccrs})

ax.gridlines(draw_labels=True,
             linewidth=2, color='gray', alpha=0.5, linestyle='--')

ax_im_bar = ax.imshow(clean_data, origin='upper', extent=mat_extent, transform=myccrs,
                      cmap=mpl.colors.LinearSegmentedColormap.from_list("mypalette", colors, N=1000))

china_boundary_gdf_sub.boundary.plot(ax=ax, color='black', linewidth=1)

fig.colorbar(ax_im_bar, orientation='vertical')
# plt.tight_layout()
#
# ax.set_title("china dem")
# ax.set_ylabel("y label")
# ax.set_xlabel("x label")

In [11]:
np.nanmin(clean_data), np.nanmax(clean_data)

(46.0, 8668.0)

## 别的投影

In [12]:
longitude = np.array(
    [getattr(out_transform, 'c') + i * getattr(out_transform, 'a') for i in range(clean_data.shape[1])])
latitude = np.array(
    [getattr(out_transform, 'f') + i * getattr(out_transform, 'e') for i in range(clean_data.shape[0])])

colors = ["#33A02C", "#B2DF8A", "#FDBF6F", "#1F78B4", "#999999", "#E31A1C", "#E6E6E6", "#A6CEE3"]
center_china_point = china_boundary_gdf.centroid[0]

# 新的兰伯特投影
new_crs = ccrs.Orthographic(central_longitude=center_china_point.x,
                                central_latitude=center_china_point.y)
myccrs = ccrs.PlateCarree()

fig, ax = plt.subplots(figsize=(10, 10), dpi=100, subplot_kw={'projection': new_crs})

ax.gridlines(draw_labels=True,
             linewidth=2, color='gray', alpha=0.5, linestyle='--')
ax_im_bar = ax.contourf(longitude, latitude, clean_data,  # [::-1,:]
                            np.arange(0, np.nanmax(clean_data), 100.0),
                            transform=ccrs.PlateCarree(),
                            cmap=mpl.colors.LinearSegmentedColormap.from_list("mypalette", colors, N=1000),
                            extend='both', origin='upper')
china_boundary_gdf_sub.to_crs(new_crs.proj4_init).boundary.plot(ax=ax, edgecolor='black', linewidth=1)
fig.colorbar(ax_im_bar, orientation='vertical')
# fig.savefig('test1226.png')
fig.savefig(f"test1226_china_{datetime.datetime.now().strftime('%Y-%m-%d %X')}.png")

/var/folders/rn/1sl8q5g56bb377glk8qyp_k40000gn/T/ipykernel_685/3706497031.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_china_point = china_boundary_gdf.centroid[0]
